In [ ]:
### trf/__init__.py ###
import ZODB, ZODB.FileStorage
import transaction
import logging
from logging.handlers import TimedRotatingFileHandler

def init_db(db_file):
    """
    Initialize the ZODB database using the specified file.
    """
    storage = ZODB.FileStorage.FileStorage(db_file)
    db = ZODB.DB(storage)
    connection = db.open()
    root = connection.root()
    return db, connection, root, transaction

def close_db(db, connection):
    """
    Close the ZODB database and its connection.
    """
    connection.close()
    db.close()

def setup_logging(trf_home, log_level=logging.INFO, backup_count=7):
    """
    Set up logging with daily rotation and a specified log level.

    Args:
        logfile (str): The file where logs will be written.
        log_level (int): The log level (e.g., logging.DEBUG, logging.INFO).
        backup_count (int): Number of backup log files to keep.
    """
    backup_count = 7

    logfile = os.path.join(trf_home, "logs", "trf.log")

    # Create a TimedRotatingFileHandler for daily log rotation
    handler = TimedRotatingFileHandler(
        logfile, when="midnight", interval=1, backupCount=backup_count
    )

    # Set the suffix to add the date and ".log" extension to the rotated files
    handler.suffix = "%y%m%d.log"

    # Create a formatter
    formatter = logging.Formatter(
        fmt='--- %(asctime)s - %(levelname)s - %(module)s.%(funcName)s\n    %(message)s',
        datefmt="%y-%m-%d %H:%M:%S"
    )

    # Set the formatter to the handler
    handler.setFormatter(formatter)

    # Define a custom namer function to change the log file naming format
    def custom_namer(filename):
        # Replace "tracker.log." with "tracker-" in the rotated log filename
        return filename.replace("track.log.", "track")

    # Set the handler's namer function
    handler.namer = custom_namer

    # Get the root logger
    logger = logging.getLogger()
    logger.setLevel(log_level)

    # Clear any existing handlers (if needed)
    if logger.hasHandlers():
        logger.handlers.clear()

    # Add the TimedRotatingFileHandler to the logger
    logger.addHandler(handler)

    logger.info("Logging setup complete.")
    logging.info(f"\n### Logging initialized at level {log_level} ###")

    return logger

### trf/__main__.py ###
from .trf import main

if __name__ == "__main__":
    main()

### trf/trf.py ###
import sys, os
from prompt_toolkit.application import Application
from prompt_toolkit.widgets import TextArea
from . import init_db, close_db, setup_logging

def process_arguments():
    """
    Process sys.argv to get the necessary parameters, like the database file location.
    """
    backup_count = 7
    log_level = logging.INFO

    if len(sys.argv) > 1:
        try:
            log_level = int(sys.argv[1])
            sys.argv.pop(1)
        except ValueError:
            print(f"Invalid log level: {sys.argv[1]}. Using default INFO level.")
            log_level = logging.INFO

    envhome = os.environ.get('TRFHOME')
    if len(sys.argv) > 1:
        trf_home = sys.argv[1]
    elif envhome:
        trf_home = envhome
    else:
        trf_home = os.getcwd()

    restore = len(sys.argv) > 2 and sys.argv[2] == 'restore'

    if len(sys.argv) < 2:
        print("Usage: track.py <db_file>")
        sys.exit(1)

    db_file = sys.argv[1]  # The first argument is the database file
    return trf_home, log_level, restore

def run_app(db_root):
    """
    Run the prompt_toolkit full-screen app.
    """
    textarea = TextArea(text="Welcome to the tracker app! Press Ctrl-C to exit.")
    app = Application(full_screen=True, layout=textarea)

    # Access the database root and interact with it here...
    print(f"Database contains: {db_root.keys()}")  # Example of accessing the db root

    # Run the application
    app.run()

def main():
    # Get command-line arguments: Process the command-line arguments to get the database file location
    trf_home, log_level, restore = process_arguments()

    # Set up logging
    logger = setup_logging(trf_home=trf_home, log_level=log_level)

    # Initialize the ZODB database

    db_file = os.path.join(trf_home, "trf.fs")

    db, connection, db_root, transaction = init_db(db_file)

    try:
        # Step 3: Run the prompt_toolkit app
        run_app(db_root)
    finally:
        # Step 4: Close the database connection when the app exits
        close_db(db, connection)

if __name__ == "__main__":
    main()

### setup.py ###
from setuptools import setup, find_packages

setup(
    name="trf-dgraham",  # Replace with your app's name
    version='0.0.1',
    author="Daniel A Graham",  # Replace with your name
    author_email="dnlgrhm@gmail.com",  # Replace with your email
    description="This is a simple application for recording the sequence of occasions on which a task is completed and forecasting when the next completion might be needed.",
    long_description=open("README.md").read(),  # If you have a README file
    long_description_content_type="text/markdown",
    url="https://github.com/dagraham/trf-dgraham",  # Replace with the repo URL if applicable
    packages=find_packages(),
    py_modules=["trf"],
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",  # Replace with your license
        "Operating System :: OS Independent",
    ],
    python_requires=">=3.7.3",  # Specify the minimum Python version
    install_requires=[
        'prompt-toolkit>=3.0.24',
        'ruamel.yaml>=0.15.88',
        'python-dateutil>=2.7.3',
    ],
    entry_points={
        'console_scripts': [
            'trf=trf.trf:main',  # Correct the path to `main` in `trf/trf.py`
        ],
    },
)

